<a href="https://colab.research.google.com/github/MarinaChau/IASD_classes/blob/master/Part_2_Single_cell_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install libraries
!pip install scprep
!pip install tensorflow==1.12

ERROR: Could not find a version that satisfies the requirement tensorflow==1.12 (from versions: 1.13.1, 1.13.2, 1.14.0, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 1.15.5, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0rc0, 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0rc0, 2.3.0rc1, 2.3.0rc2, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0rc0, 2.4.0rc1, 2.4.0rc2, 2.4.0rc3, 2.4.0rc4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.8.0rc0, 2.8.0rc1, 2.8.0)
ERROR: No matching distribution found for tensorflow==1.12


# Part 2. Analyses of datasets using unsupervised approach

This is a dataset describing development of a dentate gyrus, a part of the mouse brain. Each object is a brain cell
characterized by the expression of ~10000 genes. The cells are labeled into 24 distinct cell types.

 In this notebook, we will use different libraries to visualize our dataset.

In [9]:
## Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import io
### Third-party libraries

## Exercice
* 1) Choose one of the datasets 1-3 or suggest you own
* 2) Check if you have missing values and suggest a strategy for imputing them
* 3) Check if the dataset needs to be normalized in a particular way
* 4) Give an estimate of intrinsic dimensionality of the dataset (if you want you can use skdim package
* https://github.com/j-bac/scikit-dimension/tree/master/skdim)
* 5) Apply PCA and visualize the dataset in the space of two first principal components
* 6) Investigate higher-order principal components, try to conclude if they contain useful information
* 7) Apply any non-linear manifold learning method and visualize the dataset
* 8) Apply two or more different clustering methods using clustering quality criteria, compare the results as
* well as computational time
* 9) Apply any matrix factorization method which is not PCA, compare it with PCA as well as computational time

In [ ]:
## Q.1 Import dataset

In [4]:
from google.colab import files
uploaded = files.upload()

Saving dentate_gyrus_sample3000.csv to dentate_gyrus_sample3000.csv


In [12]:
dataframe = pd.read_csv(io.StringIO(uploaded["dentate_gyrus_sample3000.csv"].decode("utf-8")))
dataframe.head()

,label,Tmsb10,Apoe,Mt1,Stmn2,Tuba1a,Stmn1,Meg3,Ptn,Mt3,Tubb2b,Fabp7,Cst3,Xist,Sox11,Tubb3,Pcp4,Igfbpl1,Slc1a2,Slc1a3,Tubb5,Snca,Plp1,Atp1a2,Dbi,Nnat,Cd24a,Mt2,Ppp3ca,Ncdn,Gap43,Mllt11,Marcksl1,Aldoc,Camk2a,Plppr4,Clu,Glul,Ptprz1,Sparcl1,...,Shf,2610037D02Rik,Atxn7l2,Tmem74b,Adnp2,Arsg,Esco2,Tmtc3,Rnmtl1,Pald1,Dcp1a,Gemin2,Nckipsd,Notch3,Ubtd1,Simc1,Nkrf,Gabpb2,Fbxo10,Cenpl,Wdr36,Invs,Trim39,Ttc21b,Penk,Zfp629,D130020L05Rik,Pnpt1,Ice2,Pifo,Nol10,Gm6356,Tpmt,Gba,Slc25a20,Col19a1,Zfp516,Rgp1,Col26a1,Jpx
0,Neuroblast,4.47,0.00,0.35,3.09,4.62,3.72,4.04,0.19,0.19,3.81,0.19,1.43,3.4,2.30,3.01,0.00,0.99,0.00,0.00,3.63,1.58,0.00,0.00,0.49,3.83,2.08,0.19,1.62,0.35,1.88,3.01,3.59,0.19,0.00,0.62,0.00,0.72,0.35,0.00,...,0.0,0.00,0.00,0.19,0.00,0.00,0.0,0.0,0.0,0.0,0.19,0.19,0.0,0.00,0.19,0.0,0.19,0.00,0.19,0.00,0.00,0.19,0.0,0.0,0.0,0.19,0.00,0.00,0.35,0.0,0.19,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.49,0.00
1,Immature-GC,2.36,0.36,0.84,1.40,1.76,0.84,2.36,0.00,2.65,1.29,0.36,1.29,0.0,1.40,0.63,3.27,0.00,1.01,0.00,0.84,2.96,1.01,0.00,0.63,0.63,0.00,0.36,3.54,2.32,0.00,2.68,2.08,0.00,2.44,2.51,0.00,0.00,0.63,1.29,...,0.0,0.00,0.00,0.00,0.63,0.00,0.0,0.0,0.0,0.0,0.00,0.36,0.0,0.00,0.63,0.0,0.00,0.00,0.36,0.00,0.63,0.00,0.0,0.0,0.0,0.00,0.36,0.36,0.00,0.0,0.36,0.0,0.0,0.63,0.00,0.0,0.0,0.36,0.00,0.00
2,GC-juv,1.71,5.08,2.98,0.52,2.95,1.31,0.75,3.97,3.32,1.57,4.53,4.22,0.0,1.18,0.37,0.20,0.75,1.62,3.15,1.37,0.37,0.20,1.92,4.98,2.72,0.37,2.41,1.37,0.00,0.00,0.75,2.23,2.82,0.00,0.52,2.28,1.75,2.35,2.87,...,0.0,0.00,0.00,0.00,0.20,0.00,0.0,0.0,0.0,0.2,0.20,0.00,0.2,0.20,0.00,0.2,0.00,0.00,0.37,0.20,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.2,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.20,0.00,0.00
3,CA3-Pyr,4.29,0.20,0.20,3.35,4.26,3.88,3.82,1.24,0.00,3.52,0.37,1.47,1.1,1.10,2.85,3.23,1.70,0.20,0.00,4.03,1.02,0.00,0.00,1.61,2.90,2.10,0.00,1.17,0.37,3.10,2.84,3.09,0.00,0.00,0.00,0.20,0.51,0.20,0.00,...,0.2,0.20,0.20,0.00,0.00,0.20,0.0,0.2,0.2,0.0,0.00,0.00,0.0,0.00,0.20,0.2,0.00,0.00,0.00,0.00,0.20,0.00,0.2,0.2,0.0,0.00,0.00,0.20,0.00,0.0,0.00,0.2,0.0,0.00,0.20,0.0,0.0,0.00,0.37,0.00
4,GC-juv,1.91,4.08,3.24,0.41,2.67,1.01,1.10,4.69,3.58,1.79,5.36,4.31,0.0,0.41,0.22,1.18,0.22,2.44,3.25,1.79,0.92,0.22,2.83,4.38,1.39,0.81,2.82,1.18,0.00,0.56,0.41,2.44,2.77,0.00,0.22,1.10,2.58,2.83,2.98,...,0.0,0.22,0.22,0.22,0.00,0.22,0.0,0.0,0.0,0.0,0.22,0.00,0.0,0.22,0.41,0.0,0.00,0.22,0.00,0.22,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.22,0.22,0.0,0.0,0.00,0.00,0.41


In [17]:
print(f'Dataset has {dataframe.shape[0]} cells and {dataframe.shape[1]} features')

Dataset has 3000 cells and 10001 features


In [19]:
dataframe.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Columns: 10001 entries, label to Jpx
dtypes: float64(10000), object(1)
memory usage: 228.9+ MB


# Preparing data

### Q.2 Check if you have missing values and suggest a strategy for imputing them

We can either check if there is any missing value for each cell by searching NaN per row or check if there is any missing value per features by checking each row.

It depends on what we want for our dataset. If there are features with a lot of missing elements, we can either impute values or choose to delete that features. As 10 000 is a huge number of features, and we will work on reducing its number, if one feature is missing, we can neglect it (well if we consider each feature having the same weight importance). 

Another solution would be to imput data, however it requires some domain experience.


In [28]:
# Percentages of missing values
print((dataframe.isnull().mean() * 100).sort_values(ascending = True))

label      0.0
Fam135a    0.0
Abcf2      0.0
Mfsd11     0.0
Ascc1      0.0
          ... 
Dbndd2     0.0
Map2k4     0.0
Chmp2b     0.0
Rassf8     0.0
Jpx        0.0
Length: 10001, dtype: float64


In [21]:
# Searching missing values per row

In [22]:
# Searching missing values per columns